# S1 Introdução
## Apache Airflow
- Open Source - Criado pelo Airbnb em 2015. Mantido pela fundação Apache.
- Desenvolvido em Python
- Extensível

### É um orquestrador:
- Airflow não processa dados, ele coordena o processamento
- Orquestra workloads (principalmente batch)
    - Sensores e operadores "deferrable" -> permite `near real time`
- O processamento ocorre em:
    - Sistema Operacional
    - Banco de Dados
    - Spark
    - Elastic Search
    - Etc.
- Interfaces:
    - Web
    - Rest API
    - CLI
- **DAG: estrutura básica e principal do Airflow, é um pipeline**
- Um pipeline é composto por tasks que são partes dele, possui características como:
    - Precedência
    - Parelismo
    - Independência

## Pipeline:
- Erros
- Log/Auditoria
- Monitoramento/Alertas
- Recuperação a partir de um Ponto
- Dados Históricos/ Diferenciação
- Alta disponibilidade
- Distribuição de Carga"

# Como funciona o Airflow:
- DAG é um pipeline do Airflow
    - O que este pipeline vai fazer ?
- Formada por Operador e Task
- **Arquitetura**:
    - Webserver (UI)
    - Scheduler
    - Executor
    - Metadata DB
    - Triggerer
    - Queue
- `DAG = Directed acyclic graph: Grafo **direcionado** acíclico` 
    - Grafo: Conjunto de Vértices e Arestas
    - Direcionado: Possui Direção Específica
    - Acíclico: Não Circular
    - DAG é um script python
    - Não precisa ter a lógica (classe externa)
    - Um mesmo script pode ter múltiplas DAGs
    - Uma DAG pode ter uma ou mais tasks
    - Tasks possuem dependência
    - Um arquivo DAG pode ter uma ou mais de duas DAGs

- Operators: definição do trabalho:
    - Definem o 'o que fazer' no pipeline
    - Derivam de **BaseOperator**
    - Instanciados na DAG viram **Tasks**
- Tasks: Instância de um operador na DAG
- Interação com outras ferramentas:
    - AWS
    - Databricks
    - MySQL
    - MongoDB

# S2 Preparando o ambiente:

- Pré-requisitos:
    - Instalação Docker
    - IDE:
        - VSCODE
    - Instalar Airflow com Docker

- Instalar Airflow com Docker
    - 1 Crie uma pasta "airflow"
    - 2 Salve os seguintes arquivos na pasta:
        - docker-compose.yaml:
            - `C:\Users\MSILV221\classes\Forma-o-Completa-Apache-Airflow-3.1\material\Instalação\docker-compose.yaml`
        - .env
    - 3 Siga o procedimento no próximo vídeo

# S4 Estrutura básica de uma DAG

```python
###### MÓDULOS ######
import pendulum
from airflow import DAG
from airflow.operators.bash import BashOperator

###### DAG ######
# Tudo dentro do contexto da DAG use a DAG de forma automática
# Reaproveitamento de código
`O nome da DAG não é o nome do .py !`
with DAG(
    dag_id = "meu_primeiro_dag",
    description = "Minha primeira DAG no Airflow",
    schedule = None, # Sem agendamento
    start_date=pendulum.datetime(2024, 1, 1, tz="America/Sao_Paulo"),
    catchup = False, # Se a DAG falhar ele executa execuções passadas que ficaram pendentes
    tags=["curso", "exemplo"],
) as dag:
###### TASKS ######
    task1 = BashOperator(
        task_id = "task1",
        bash_command = "sleep 5",
    ),
    task2 = BashOperator(
        task_id = "task2",
        bash_command = "echo 'Hello, Airflow!'",
    )
    task3 = BashOperator(
        task_id = "task3",
        bash_command = "date",
    )
##### Ordem de precedência / Execução ######
    task1 >> task2 >> task3 # Indica a ordem de execução
```